##### Imports & Read Data

In [173]:
import json
import pandas as pd
import numpy as np
import re
from calendar import *
from datetime import datetime, date, time

data = pd.read_json('.\Spotify Account Data\StreamingHistory0.json')

##### Data Ingestion & Overview

In [2]:
data.head()

,endTime,artistName,trackName,msPlayed
0,2022-12-20 18:44,James O'Brien - The Whole Show,A dog has crashed a car,2817984
1,2022-12-21 17:41,Up First,"Wednesday, December 21st, 2022",912746
2,2022-12-21 17:56,The NPR Politics Podcast,"Jan. 6 ""Ringleader"": Lawmakers Advise Criminal...",884906
3,2022-12-21 21:37,James O'Brien - The Whole Show,A dog has crashed a car,6059785
4,2022-12-22 13:59,James O'Brien - The Whole Show,Some calls stay with you forever,8904468


In [3]:
# Reordr columns for readability

data = data[['artistName', 'trackName', 'endTime',  'msPlayed']]
data.head()

,artistName,trackName,endTime,msPlayed
0,James O'Brien - The Whole Show,A dog has crashed a car,2022-12-20 18:44,2817984
1,Up First,"Wednesday, December 21st, 2022",2022-12-21 17:41,912746
2,The NPR Politics Podcast,"Jan. 6 ""Ringleader"": Lawmakers Advise Criminal...",2022-12-21 17:56,884906
3,James O'Brien - The Whole Show,A dog has crashed a car,2022-12-21 21:37,6059785
4,James O'Brien - The Whole Show,Some calls stay with you forever,2022-12-22 13:59,8904468


In [4]:
data.shape

(8276, 4)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8276 entries, 0 to 8275
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artistName  8276 non-null   object
 1   trackName   8276 non-null   object
 2   endTime     8276 non-null   object
 3   msPlayed    8276 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 258.8+ KB


In [6]:
tbl1 = data.nunique(axis=0)
print(tbl1)

artistName     577
trackName     2528
endTime       7746
msPlayed      4115
dtype: int64


##### Clean Data & Format Datetime

In [184]:
# Convert ms > hh:mm:ss
t_col = []
for i in data['msPlayed']:
    seconds = (i/1000)%60
    minutes = (i/(1000*60)%60)
    hours = (i/(1000*60*60))%24
    t_col.append('%02d:%02d:%02d' %(hours, minutes, seconds))

data['Runtime'] = t_col
# data['Runtime (mins)'] = data['msPlayed'] / 60000
# data['Runtime (mins)'] = data['Runtime (mins)'].round(2)

data.head()

,Artist,Track,Runtime,Date,Time,Type,Day,DateTime,msPlayed
0,James O'Brien - The Whole Show,A dog has crashed a car,00:46:57,"Tue Dec 20, 22",01:44 PM,Podcast,Tuesday,2022-12-20 13:44:00-05:00,2817984
1,Up First,"Wednesday, December 21st, 2022",00:15:12,"Wed Dec 21, 22",12:41 PM,Podcast,Wednesday,2022-12-21 12:41:00-05:00,912746
2,The NPR Politics Podcast,"Jan. 6 ""Ringleader"": Lawmakers Advise Criminal...",00:14:44,"Wed Dec 21, 22",12:56 PM,Podcast,Wednesday,2022-12-21 12:56:00-05:00,884906
3,James O'Brien - The Whole Show,A dog has crashed a car,01:40:59,"Wed Dec 21, 22",04:37 PM,Podcast,Wednesday,2022-12-21 16:37:00-05:00,6059785
4,James O'Brien - The Whole Show,Some calls stay with you forever,02:28:24,"Thu Dec 22, 22",08:59 AM,Podcast,Thursday,2022-12-22 08:59:00-05:00,8904468


In [176]:
# Generate a column for day of the week - for binning, could probably use the eventual 'Date' column, but...whatever

data['Day'] = pd.to_datetime(data['endTime'])

data['Day'] = data['Day'].dt.day_name()

data['endTime'] = pd.to_datetime(data['endTime'])

data.head()

,endTime,artistName,trackName,msPlayed,Runtime,Day
0,2022-12-20 18:44:00,James O'Brien - The Whole Show,A dog has crashed a car,2817984,0:46:57,Tuesday
1,2022-12-21 17:41:00,Up First,"Wednesday, December 21st, 2022",912746,0:15:12,Wednesday
2,2022-12-21 17:56:00,The NPR Politics Podcast,"Jan. 6 ""Ringleader"": Lawmakers Advise Criminal...",884906,0:14:44,Wednesday
3,2022-12-21 21:37:00,James O'Brien - The Whole Show,A dog has crashed a car,6059785,1:40:59,Wednesday
4,2022-12-22 13:59:00,James O'Brien - The Whole Show,Some calls stay with you forever,8904468,2:28:24,Thursday


In [177]:
# Times are given in UTC, so next I'm going to convert them to EST

# localize
data['endTime'] = data['endTime'].dt.tz_localize('UTC')

#convert
data['endTime'] = data['endTime'].dt.tz_convert('US/Eastern')
data.head()

,endTime,artistName,trackName,msPlayed,Runtime,Day
0,2022-12-20 13:44:00-05:00,James O'Brien - The Whole Show,A dog has crashed a car,2817984,0:46:57,Tuesday
1,2022-12-21 12:41:00-05:00,Up First,"Wednesday, December 21st, 2022",912746,0:15:12,Wednesday
2,2022-12-21 12:56:00-05:00,The NPR Politics Podcast,"Jan. 6 ""Ringleader"": Lawmakers Advise Criminal...",884906,0:14:44,Wednesday
3,2022-12-21 16:37:00-05:00,James O'Brien - The Whole Show,A dog has crashed a car,6059785,1:40:59,Wednesday
4,2022-12-22 08:59:00-05:00,James O'Brien - The Whole Show,Some calls stay with you forever,8904468,2:28:24,Thursday


In [181]:
# Create easily readable time column, note this is a string, time operations should always be performed on the full datetime column

data['Time'] = None

data['Time'] = data['endTime'].dt.strftime("%I:%M %p") # .strftime formats the datetime as a string

########################################################

data['Date'] = None

data['Date'] = data['endTime'].dt.strftime("%a %b %d, %y")

data

# This step creates a lot of redundancy. The strings are readable in the table, the 'endTime' column is for timeseries operations and the Day column is for binning.

,endTime,artistName,trackName,msPlayed,Runtime,Day,Time,Date
0,2022-12-20 13:44:00-05:00,James O'Brien - The Whole Show,A dog has crashed a car,2817984,0:46:57,Tuesday,01:44 PM,"Tue Dec 20, 22"
1,2022-12-21 12:41:00-05:00,Up First,"Wednesday, December 21st, 2022",912746,0:15:12,Wednesday,12:41 PM,"Wed Dec 21, 22"
2,2022-12-21 12:56:00-05:00,The NPR Politics Podcast,"Jan. 6 ""Ringleader"": Lawmakers Advise Criminal...",884906,0:14:44,Wednesday,12:56 PM,"Wed Dec 21, 22"
3,2022-12-21 16:37:00-05:00,James O'Brien - The Whole Show,A dog has crashed a car,6059785,1:40:59,Wednesday,04:37 PM,"Wed Dec 21, 22"
4,2022-12-22 08:59:00-05:00,James O'Brien - The Whole Show,Some calls stay with you forever,8904468,2:28:24,Thursday,08:59 AM,"Thu Dec 22, 22"
...,...,...,...,...,...,...,...,...
8271,2023-12-21 15:31:00-05:00,Megan Thee Stallion,Savage Remix (feat. Beyoncé),242000,0:4:2,Thursday,03:31 PM,"Thu Dec 21, 23"
8272,2023-12-21 15:35:00-05:00,Hozier,Eat Your Young,243946,0:4:3,Thursday,03:35 PM,"Thu Dec 21, 23"
8273,2023-12-21 15:39:00-05:00,Yebba,The Age of Worry - Live at Electric Lady,227880,0:3:47,Thursday,03:39 PM,"Thu Dec 21, 23"
8274,2023-12-21 15:43:00-05:00,Charlotte Day Wilson,Take Care of You (feat. Syd),214437,0:3:34,Thursday,03:43 PM,"Thu Dec 21, 23"


In [178]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8276 entries, 0 to 8275
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype                     
---  ------      --------------  -----                     
 0   endTime     8276 non-null   datetime64[ns, US/Eastern]
 1   artistName  8276 non-null   object                    
 2   trackName   8276 non-null   object                    
 3   msPlayed    8276 non-null   int64                     
 4   Runtime     8276 non-null   object                    
 5   Day         8276 non-null   object                    
dtypes: datetime64[ns, US/Eastern](1), int64(1), object(4)
memory usage: 388.1+ KB


##### Categorize & Lable Type

In [13]:
artists = data['artistName'].unique().tolist()
len(artists)

577

In [58]:
# List of podcasts generated from last year

podcasts_v1 = ['1A','FiveThirtyEight Politics','The NPR Politics Podcast','Throughline','The Professional Chatterbox',
'You\'re Wrong About','Code Switch','Fresh Air','Beyond the Scenes from The Daily Show','Harry, Meghan and the Media',
'Off The Rails','Gossip with Celebitchy','The Royal Report','Bryony Gordon\'s Mad World','Higher Learning with Van Lathan and Rachel Lindsay',
'Disclosure','The Rachel Maddow Show','Archewell Audio','By The Book','The Man Enough Podcast','Offline with Jon Favreau',
'Maintenance Phase','Armchair Expert with Dax Shepard','Conversations with People Who Hate Me','Sounds Like A Cult',
'Masters of Scale','Queens Cast''The Read','All In with Chris Hayes','QAnon Anonymous','Swindled','Life After MLM',
'Fit Literate','The Up-Beet Dietitians','Con Artists','Pod Save America','Up First','All Songs Considered',
'Pod Save the World','Downline','Into It: A Vulture Podcast with Sam Sanders', 'Criminal','The Ezra Klein Show',
'How It Happened','Truthers: Tiffany Dover Is Dead*','Notes from America','The Receipts Podcast', 'The Cut',
'The New Way We Work','Embedded', 'Pop Culture Happy Hour','Mythology','Archetypes','Larry Wilmore: Black on the Air',
'Louder Than A Riot','You and Me Both with Hillary Clinton','Dates & Mates with Damona Hoffman','The Wilderness',
 'Make Me Smart', 'Bloomberg Businessweek','The Inquiry','The Gal Code','Strict Scrutiny', 'Hard Fork', "James O'Brien - The Whole Show",
 "I'ma Need More Wine Podcast",'Metro Morning from CBC Radio Toronto (Highlights)','CBC News At Issue','Front Burner','The House from CBC Radio',
 'The Daily Show With Trevor Noah: Ears Edition','If Books Could Kill','Around The Way Curls','On with Kara Swisher',
 'Why Is This Happening? The Chris Hayes Podcast','Fit Literate','Con Artists ','The Stoop','Jordan Klepper Fingers the Conspiracy','Vibe Check','Pivot','BYLINE TIMES PODCAST' 
 ]


len(podcasts_v1)

In [60]:
# List of music artists generated from last years data

music_v1 = ['Megan Thee Stallion','Chlöe','Jesy Nelson','SZA','Ibeyi','Dave',
 'Stormzy','Burna Boy','Jorja Smith','Master KG','BNXN fka Buju','L.A.X',
 'Khalid','Beyoncé','Wizkid','SPINALL','Olamide','Mr Eazi','melvitto',
 'Praiz','J Hus','Rotimi','Koffee','Etta James','Little Mix','Big Sean',
 'Rihanna','DJ Khaled','Nicki Minaj','Panic! At The Disco','WALK THE MOON',
 'The Carters','Ciara','Cardi B','Eminem','Rick Ross','JAY-Z','Kanye West',
 'Sam Smith','The Script','Paramore','Adele','Lewis Capaldi',
 'Billie Eilish','Jon Batiste','Sigala','Tori Kelly','Mariah Carey',
 'Frank Sinatra','Bobby Helms','Eartha Kitt','Johnny Mathis',
 'Harry Belafonte','Bing Crosby','Ray Conniff','Perry Como',
 'Victor Herbert','Andy Williams','Ella Fitzgerald','Dean Martin',
 'Tina Turner','Bruce Springsteen','The Police','Culture Club',
 'Michael Jackson','Whitney Houston','Bryan Adams','Earth, Wind & Fire',
 'Aretha Franklin','Randy Crawford','Diana Ross','Luther Vandross',
 'Lionel Richie','Natalie Cole','Deniece Williams','Chaka Khan',
 'Cyndi Lauper','Phil Collins','Salatiel','Blue Ivy','Tiwa Savage','Nija',
 '070 Shake','Lady Gaga','Sia','Demi Lovato','Olivia Rodrigo','Bruno Mars',
 'James Bay','Katy Perry','Shawn Mendes','Ed Sheeran','Amanda Seyfried',
 'Taylor Swift','Becky Hill','James Arthur','ZAYN','Coldplay',
 'One Direction','Fleetwood Mac','Marshmello','Justin Bieber',
 'Post Malone','WATTS','Kendrick Lamar','Calvin Harris','Lorde',
 'Louis The Child','5 Seconds of Summer','benny blanco','Bazzi','Giveon',
 'Daniel Caesar','Jessie Reyez','Tate McRae','Kane Brown','MEDUZA','Lizzo',
 'Kelly Rowland','Nelly','Jazmine Sullivan','Ariana Grande',
 'Chloe x Halle','DJ Snake','Maroon 5','Lil Nas X','Gunna',
 'Curtis Harding','Ramsey','Imagine Dragons','Zendaya','Leon Bridges',
 'Bon Iver','Foy Vance','Blanco White','Mipso','Aisha Badru','Haux',
 'Madi Diaz','Caamp','Robert Plant','Villagers','Shannon Lay','Lord Huron',
 'Dermot Kennedy','Amos Lee','Noah Gundersen','The Lumineers',
 'Emily Scott Robinson','Damien Rice',"Aoife O'Donovan" 'Ray LaMontagne',
 'Brandi Carlile','Brett Dennen','Tracy Chapman','Bahamas','Ben Harper',
 'Eric Hutchinson','Ryan Adams','The Head And The Heart','Norah Jones',
 'Corinne Bailey Rae','Simply Red','Alanis Morissette','Sade',
 'James Morrison','Chris Stapleton','Amber Gray','André De Shields',
 'Eva Noblezada','Hadestown Original Broadway Company','Reeve Carney',
 'Patrick Page','Jewelle Blackman','Hadestown Original Broadway Band',
 'Christine and the Queens','Selena Gomez','Jason Mraz','Yseult',
 'Harry Styles','Ber','Camila Cabello','Calum Scott','Lizzy McAlpine',
 'Halsey','Zevia','Hillsong UNITED','Justin Timberlake','Sara Bareilles',
 'Kurt Cobain','Jenna Raine','Iron & Wine','Charlie Puth','Bruno Major',
 'Frank Ocean','X Ambassadors','Jessie Murph','Kelly Clarkson',
 'Alessia Cara','Novo Amor','GAYLE','Summer Walker','John Vincent III',
 'Emeli Sandé','Amy Winehouse','Montell Fish','Sam Fischer','H.E.R.',
 'Noah Cyrus','Kina Grannis','Kimbra','Gotye','Janelle Monáe','Solange',
 'Samuel E. Wright','Sara Ramirez','Macklemore & Ryan Lewis','Halestorm',
 'Michael Bublé','RILEY','Jessie Ware','Brandy','Kurt Hugo Schneider',
 'J. Cole','Anastacia','Ben Moody','Marvin Gaye','Bill Withers',
 'Sam Cooke','Otis Redding','Donny Hathaway','Lou Rawls','Hozier',
 'Ms. Lauryn Hill','Florence + The Machine','The Blues Brothers',
 'Chris Botti','Labrinth','Matt Corby','Masego','Sonder','The Weeknd',
 'Fireboy DML','Conan Gray','PJ Morton','Paloma Faith','Lana Del Rey',
 'Snoh Aalegra','LÉON','Yebba','Teyana Taylor','John Legend',
 'Lauren Spencer Smith','Van Morrison','Mahalia Jackson','Clinton Kane',
 'Loren Allred','Glee Cast','Meghan Trainor','A Fine Frenzy','Common',
 'The Civil Wars','India.Arie','Nina Simone','THE ANXIETY',
 'Yusuf / Cat Stevens','B.B. King','OneRepublic','Anoushka Shankar',
 'Ry Cuming','Machine Gun Kelly','Tom Walker','Sigma','Ofenbach',
 'Ella Henderson','Kygo','Bugzy Malone','Alicia Keys','Nathan Sykes',
 'Dominic Fike','Zac Efron','Hugh Jackman','Bonnie Tyler','James Blake',
 'Mumford & Sons','Jerry Douglas','Ben Howard','Passenger','Normani',
 'Gerry Rafferty','Allen Stone','Tove Lo','Honey Dijon','Lil Yamaha',
 'Jhené Aiko','Kehlani',"Rag'n'Bone Man" 'JoJo','Jill Scott','Jess Benko',
 'Triple One','Mac Miller','Jack Harlow','Ty Dolla $ign','FKA twigs',
 'Fivio Foreign','Drake','Lucky Daye','Lykke Li','Mark Ronson',
 'Lily Allen','Patrick Watson','The Roots','UltraViolet DJs',
 'The Neighbourhood','Syd','Tyler, The Creator','The Internet',
 'Parker Louis','Mary J. Blige','Nature Sounds','Mahalia','Aloe Blacc',
 'Sabrina Claudio','G-Eazy','WHIPPED CREAM','Doja Cat','Saweetie',
 'Charlotte Lawrence','Jucee Froot','Maisie Peters','Cyn','K.Flay',
 'Lauren Jauregui','Sofi Tukker','Jurnee Smollett-Bell','Raveena',
 'Galantis','Heather Headley','Jason Raize','Adam Pascal',
 'Ensemble - Aida','Jacob Banks','Jessie J','The Darkness',
 'Victoria Monét','Fifth Harmony','Ella Mai','Wild Rivers',
 'Sinéad Harnett','Jim Croce','Maze','Rudimental','Robert Glasper',
 'Khruangbin','Santana','Tanner Adell',
 'Cat Burns','Nao','Ella Eyre',"Destiny's Child" 'Michelle Williams',
 'The Cheetah Girls','Fugees','Tekno','Tems','Ronald Isley','Todrick Hall',
 'Joe Aaron Reid','Liisi Lafontaine','Teena Marie','Shara Nelson',
 'Leslie Odom Jr.','Lin-Manuel Miranda','Renée Elise Goldsberry',
 'Thayne Jasperson','Jonathan Groff','Phillipa Soo','Okieriete Onaodowan',
 'Original Broadway Cast of Hamilton','Christopher Jackson','Daveed Diggs',
 'Jasmine Cephas-Jones','Ariana DeBose','Evanescence','Tamera',
 'Jennifer Hudson','Marcus Mumford','Maths Time Joy','Latto','NEIKED',
 'Amaarae','Elton John','Cynthia Erivo','The Halluci Nation',
 'Fall Out Boy','Kings of Leon','Thirty Seconds To Mars','Ashley McBryde',
 'Vance Joy','A$AP Rocky','ROSALÍA','Matt Bellamy']


len(music_v1)

In [65]:
# Generate a list of artists/shows not in 2022 data, so I'm only sorting through new names - just realized I probably could have done some ofthis with set operators

creator_2023 = data[~data['artistName'].isin(podcasts_v1) & ~data['artistName'].isin(music_v1)]
creator_2023 # new df

In [68]:
new_creators = creator_2023['artistName'].unique().tolist()
len(new_creators)

357

In [69]:
# Manually weed out the music artists...including some weird ones from Spotify made and sleep playlists
new_creators

['Death Panel',
 'Stromae',
 'The Read',
 'The Problem With Jon Stewart',
 'JVCK JAMES',
 'Aaliyah',
 'Vivir Quintana',
 'Joseph Lawrence',
 'Brittany Howard',
 'The House',
 'Fairuz',
 'Therapy for Black Girls',
 'I Weigh with Jameela Jamil',
 'Ranky Tanky',
 'Samara Joy',
 'Fad Camp',
 'Your Undivided Attention',
 'The Prof G Pod with Scott Galloway',
 'VIC MENSA',
 'Vince Staples',
 'Smart Podcast, Trashy Books',
 'Constantine Lupo',
 'Soothe My Soul',
 'Roy Holtz',
 'Andy Kamman',
 'Zilliondust',
 'Astral Wonder',
 'Menidia',
 'Juni Ros',
 'Binaural Landscapes',
 'Cameron Sinosa',
 'Beatrice Laporte',
 'Lo Amber',
 'Sleepy Moon',
 'Aileen Morais',
 'Kamila Tait',
 'Deep Motions',
 'Ceri Rocha',
 'Elodie Acy',
 'Zone Out',
 'Oberohn',
 'Succession of Images',
 'David Ripperton',
 'Muni Yogi',
 'Cloudville',
 'The Deep Sleep Scientists',
 'Astro.Not',
 'Peaceful Village',
 'Spheres',
 'Monet Benitez',
 'Switched on Pop',
 'Ear Hustle',
 'Looking For The Todt Family',
 'Missy Elliott'

In [70]:
# New podcasts in data from 2023

podcasts_v2 = ['Death Panel',
 'The Read',
 'The Problem With Jon Stewart',
 'The House',
 'Therapy for Black Girls',
 'I Weigh with Jameela Jamil',
 'Your Undivided Attention',
 'The Prof G Pod with Scott Galloway',
 'Smart Podcast, Trashy Books',
 'Switched on Pop',
 'Ear Hustle',
 'Looking For The Todt Family',
 'The Daily',
 'All Of It',
 'For Colored Nerds',
 'Booked Up with Jen Taub',
 'How to Be Fine',
 'There Are No Girls on the Internet',
 'The Trawl',
 "Full Disclosure with James O'Brien",
 'Conspirituality',
 'The ReidOut with Joy Reid',
 'Pod Save the UK',
 'Prosecuting Donald Trump',
 'Fixing Famous People with Chris DeRosa & Dominick Pupa',
 'Daily Fail',
 'Some of My Best Friends Are',
 'Long Shadow',
 'Rachel Maddow Presents: Déjà News',
 'The Bakari Sellers Podcast',
 "Odi's Office Presents: Tha Antidote",
 'CNN One Thing',
 'theGrio Daily, Michael Harriot',
 'The Worst Bestsellers',
 'The Tall & Short of It Podcast',
 'Rethinking Wellness',
 'The Black Girl Bravado',
 'Anna Faris Is Unqualified',
 'The Data Scientist Show',
 'Kubernetes Unpacked',
 'TED Radio Hour',
 'Indestructible PR Podcast with Molly McPherson',
 'Talk TV Radio',
 'Oh God, What Now?',
 '99% Invisible',
 'Velshi']

len(podcasts_v2)

In [72]:
# Make a list for both years podcasts

pods_2024 = podcasts_v1 + podcasts_v2

In [76]:
(type(pods_2024), len(pods_2024))

(list, 127)

In [78]:
# Save the podcast list for reference next year

with open("podcasts2024_no_headder.txt", 'w') as output:
    output.write(str(pods_2024))

In [182]:
# add a 'Type' column to the table based on the list of podcasts

data['Type'] = None
data['Type'] =  np.where(data['artistName'].isin(pods_2024), 'Podcast', 'Music')

data.head()

,endTime,artistName,trackName,msPlayed,Runtime,Day,Time,Date,Type
0,2022-12-20 13:44:00-05:00,James O'Brien - The Whole Show,A dog has crashed a car,2817984,0:46:57,Tuesday,01:44 PM,"Tue Dec 20, 22",Podcast
1,2022-12-21 12:41:00-05:00,Up First,"Wednesday, December 21st, 2022",912746,0:15:12,Wednesday,12:41 PM,"Wed Dec 21, 22",Podcast
2,2022-12-21 12:56:00-05:00,The NPR Politics Podcast,"Jan. 6 ""Ringleader"": Lawmakers Advise Criminal...",884906,0:14:44,Wednesday,12:56 PM,"Wed Dec 21, 22",Podcast
3,2022-12-21 16:37:00-05:00,James O'Brien - The Whole Show,A dog has crashed a car,6059785,1:40:59,Wednesday,04:37 PM,"Wed Dec 21, 22",Podcast
4,2022-12-22 08:59:00-05:00,James O'Brien - The Whole Show,Some calls stay with you forever,8904468,2:28:24,Thursday,08:59 AM,"Thu Dec 22, 22",Podcast


In [87]:
# Check a random cross section of the df
data[20:40] # This visual check set off a day's work reconciling the various time formats presented in this df, starting with the timezone of the original datetime (see Programmer's Log)

,artistName,trackName,endTime,msPlayed,Runtime (mins),Day,Type
20,FiveThirtyEight Politics,The Politics Of Prosecuting Trump,2022-12-28 20:10:00-05:00,4983086,83.05,Wednesday,Podcast
21,Maintenance Phase,Workplace Wellness,2022-12-28 20:54:00-05:00,2505200,41.75,Wednesday,Podcast
22,Beyoncé,ALIEN SUPERSTAR,2022-12-28 21:00:00-05:00,398,0.01,Wednesday,Music
23,Beyoncé,CUFF IT,2022-12-28 21:00:00-05:00,353597,5.89,Wednesday,Music
24,Beyoncé,ENERGY (feat. Beam),2022-12-28 21:02:00-05:00,116727,1.95,Wednesday,Music
25,Beyoncé,BREAK MY SOUL,2022-12-28 21:07:00-05:00,278281,4.64,Wednesday,Music
26,Beyoncé,CHURCH GIRL,2022-12-28 21:14:00-05:00,446679,7.44,Wednesday,Music
27,Beyoncé,PLASTIC OFF THE SOFA,2022-12-28 21:17:00-05:00,161489,2.69,Wednesday,Music
28,The Roots,"My Shot (feat. Busta Rhymes, Joell Ortiz & Nat...",2022-12-28 21:22:00-05:00,13541,0.23,Wednesday,Music
29,Cardi B,Up,2022-12-28 21:22:00-05:00,310502,5.18,Wednesday,Music


In [185]:
# Rename and reorder the columns for ease of use

data = data.rename(columns={'artistName':'Artist', 'trackName': 'Track', 'Runtime': 'Runtime', 'Date':'Date', 'Time': 'Time', 'Type': 'Type','Day': 'Day', 'endTime': 'DateTime','msPlayed': 'msPlayed'})
data = data[['Artist','Track', 'Runtime', 'Date', 'Time', 'Type','Day', 'DateTime','msPlayed']]
data

,Artist,Track,Runtime,Date,Time,Type,Day,DateTime,msPlayed
0,James O'Brien - The Whole Show,A dog has crashed a car,00:46:57,"Tue Dec 20, 22",01:44 PM,Podcast,Tuesday,2022-12-20 13:44:00-05:00,2817984
1,Up First,"Wednesday, December 21st, 2022",00:15:12,"Wed Dec 21, 22",12:41 PM,Podcast,Wednesday,2022-12-21 12:41:00-05:00,912746
2,The NPR Politics Podcast,"Jan. 6 ""Ringleader"": Lawmakers Advise Criminal...",00:14:44,"Wed Dec 21, 22",12:56 PM,Podcast,Wednesday,2022-12-21 12:56:00-05:00,884906
3,James O'Brien - The Whole Show,A dog has crashed a car,01:40:59,"Wed Dec 21, 22",04:37 PM,Podcast,Wednesday,2022-12-21 16:37:00-05:00,6059785
4,James O'Brien - The Whole Show,Some calls stay with you forever,02:28:24,"Thu Dec 22, 22",08:59 AM,Podcast,Thursday,2022-12-22 08:59:00-05:00,8904468
...,...,...,...,...,...,...,...,...,...
8271,Megan Thee Stallion,Savage Remix (feat. Beyoncé),00:04:02,"Thu Dec 21, 23",03:31 PM,Music,Thursday,2023-12-21 15:31:00-05:00,242000
8272,Hozier,Eat Your Young,00:04:03,"Thu Dec 21, 23",03:35 PM,Music,Thursday,2023-12-21 15:35:00-05:00,243946
8273,Yebba,The Age of Worry - Live at Electric Lady,00:03:47,"Thu Dec 21, 23",03:39 PM,Music,Thursday,2023-12-21 15:39:00-05:00,227880
8274,Charlotte Day Wilson,Take Care of You (feat. Syd),00:03:34,"Thu Dec 21, 23",03:43 PM,Music,Thursday,2023-12-21 15:43:00-05:00,214437


##### Save Output

In [186]:
# Save the transformed data to a csv to be analyzed in a separate notebook.

folder = 'C:\\Users'
data.to_csv(folder + '\spotify_2023_clean.csv', index=True)
